In [ ]:
import requests

original_url = 'https://vnexpress.net/thoi-su'


response = requests.get(original_url)
html_content = response.text
print(html_content)


In [12]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content, 'html.parser')
h3_tags = soup.find_all('h3')

links_titles = set()
for tag in h3_tags:
    a_tag = tag.find('a')
    if a_tag:
        link = a_tag.get('href')
        title = a_tag.get('title')
        links_titles.add((link, title))

print(links_titles)

{('https://vnexpress.net/ong-nguyen-van-hieu-lam-pho-ban-tuyen-giao-trung-uong-4840268.html', 'Ông Nguyễn Văn Hiếu làm Phó ban Tuyên giáo Trung ương'), ('https://vnexpress.net/xuong-lam-linh-vat-ran-khong-lo-duong-hoa-nguyen-hue-4839436.html', 'Xưởng làm linh vật rắn khổng lồ đường hoa Nguyễn Huệ'), ('https://vnexpress.net/pho-chu-tich-tinh-dong-thap-huynh-minh-tuan-bi-khien-trach-4839117.html', 'Phó chủ tịch tỉnh Đồng Tháp Huỳnh Minh Tuấn bị khiển trách'), ('https://vnexpress.net/metro-ben-thanh-suoi-tien-se-cho-khach-xuyen-dem-giao-thua-4840477.html', 'Metro Bến Thành - Suối Tiên sẽ chở khách xuyên đêm giao thừa'), ('https://vnexpress.net/pho-bi-thu-tp-hcm-lam-bi-thu-ca-mau-4840420.html', 'Phó bí thư TP HCM làm Bí thư Cà Mau'), ('https://vnexpress.net/nhieu-chang-tau-xe-kin-cho-dip-tet-4838709.html', 'Nhiều chặng tàu, xe kín chỗ dịp Tết'), ('https://vnexpress.net/bo-tai-nguyen-moi-truong-phuong-an-cua-ha-noi-chua-lam-song-lai-song-to-lich-4839720.html', "Bộ Tài nguyên Môi trường: Phư

In [51]:
from bs4 import BeautifulSoup
import requests

def process_p_contents(p_contents):
    return " ".join(p_contents)

def scrape_data_by_cate(url):
    # Fetch and parse the main category page
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch URL: {url}, Status Code: {response.status_code}")
        return []

    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    h3_tags = soup.find_all('h3')
    
    # Store the scraped data
    scraped_data = []

    id = 0
    for tag in h3_tags:
        a_tag = tag.find('a')
        if a_tag:
            link = a_tag.get('href')
            title = a_tag.get('title')
            
            # Fetch and parse the linked page
            try:
                response_title = requests.get(link)
                if response_title.status_code != 200:
                    print(f"Failed to fetch linked page: {link}, Status Code: {response_title.status_code}")
                    continue
                
                title_soup = BeautifulSoup(response_title.text, 'html.parser')
                
                # Find all <p> tags with class "Normal"
                p_tags = title_soup.find_all('p', class_='Normal')
                p_contents = [p_tag.text.strip() for p_tag in p_tags]
                
                content = process_p_contents(p_contents)

                scraped_data.append({
                    'id': id,
                    'link': link,
                    'title': title,
                    'content': content
                })
                id += 1

            except Exception as e:
                print(f"Error fetching or parsing linked page: {link}, Error: {e}")
                continue

    return scraped_data

url = 'https://vnexpress.net/thoi-su'
data = scrape_data_by_cate(url)


In [53]:
import json

with open('scraped_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)